In [1]:
import random
import numpy as np
import pandas as pd

In [2]:
from buffer_pool_baseline.environment import Query, Cache, Time
from IPython.display import clear_output
from tqdm import tqdm

In [3]:
%matplotlib inline

In [4]:
q_table = np.zeros([1000, 2])
cache_size = 4

In [5]:
def setup():
    t = Time(0)
    
    c = Cache(cache_size, t, equate_id_to_value=True)
    
    start = random.randint(0, 20)
    end = start + 5
    loop_size = random.randint(1, 3)
    
    env = Query(query_type="sequential", time=t, parameters={"start": start, "end": end, "loop_size": loop_size})
    env.set_query_cache(c)
    
    c.add_element(random.randint(0, 30))
    c.add_element(random.randint(0, 30))
    c.add_element(random.randint(0, 30))
    c.add_element(random.randint(0, 30))
    return env

In [6]:
def get_query(query_type, table_size, loop_size):
    t = Time()
    if query_type == "sequential":
        return Query(query_type="sequential", time=t, parameters={"start": 0, "end": table_size, "loop_size": loop_size})
    
    if query_type == "select":
        return Query(query_type="select", time=t, parameters={"start": 0, "end": table_size})
    
    if query_type == "join":
        return Query(query_type="join", time=t, parameters={"start_table_1": 0, "end_table_1": 10, 
                                                            "start_table_2": 0, "end_table_2": 10})
    

In [7]:
def get_time_steps_per_query(table_size, cache_size, loop_size):
    time_steps = {}
    
    query_types = ["sequential", "select", "join"]
    
    for query in query_types:
        t = Time(0)
        env = get_query(query, table_size, loop_size)
        c = Cache(cache_size, t, equate_id_to_value=True)
        env.set_query_cache(c)

        while not env.is_done():
            env.step("mru")

        time_steps[query] = env.time.now()
    
    return time_steps

In [8]:
def encode_queries(query_type, time_step, max_time_steps):
    return max_time_steps[query_type] + time_step

In [9]:
max_time_steps = get_time_steps_per_query(10, 4, 5)

In [10]:
alpha = 0.05
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

In [11]:
q_values_cumulative = []
old_q = np.zeros([1000, 2])

In [12]:
number_of_runs = 10000

In [13]:
cum_reward_plot = []

for i in tqdm(range(1, number_of_runs)):
    
    env = setup()
    state = encode_queries(env.query_type, env.time.now(), max_time_steps)
    
    penalties, reward, = 0, 0
    done = False
    previous_hit, previous_miss = 0, 0
    cum_reward = 0
    tm = 0
    
    while not done:
        
        if random.uniform(0, 1) < epsilon:
            action = random.choice(env.actions) # Explore action space
        else:
            _action_ = np.argmax(q_table[state]) # Exploit learned values
            action = "mru" if _action_ == 0 else "lru"
            
        hits, miss = env.step(action)
        next_state = encode_queries(env.query_type, env.time.now(), max_time_steps)
        
        reward = hits - previous_hit
        penalties = miss - previous_miss
        
        previous_hit = hits
        previous_miss = miss
        
        done = env.done

        _action_ = 0 if action == "mru" else 1
        
        old_value = q_table[state, _action_]
        next_max = np.max(q_table[next_state])
        
        r = reward - (10 * penalties)
        new_value = (1 - alpha) * old_value + alpha * (r + gamma * next_max)
        q_table[state, _action_] = new_value
        state = next_state
        cum_reward += r
        tm += 1
        
    old_q = q_table.copy()
    cum_reward_plot.append({"reward": cum_reward, "epoch": i, "time": tm})

print("Training finished.\n")

100%|████████████████████████████████████████████████████████████████████████████| 9999/9999 [00:09<00:00, 1027.11it/s]

Training finished.



In [14]:
df = pd.DataFrame(q_values_cumulative)

In [15]:
max_time_steps

{'sequential': 66, 'select': 11, 'join': 111}

In [16]:
def evaluate_policy(query, q_table):
    cache_map_policy = []
    
    start_index = max_time_steps[query.query_type]
    
    reward = 0
    
    for d in q_table[start_index:]:
        action = ["mru", "lru"][np.argmax(d)]
        cache_map_policy.append({"action": action, 
                                 "cache": query.cache.cache_map.copy().keys(),
                                "which_element": query.parameters.copy()})
        query.step(action)
        
        if query.is_done():
            break
    
    hits, misses = query.step()
    reward = hits -  misses
    return reward, cache_map_policy
    

In [17]:
def get_baseline(query, action = "mru"):
    cache_map_baseline = []
    
    while not query.is_done():
        query.step(action)
        cache_map_baseline.append({"cache": query.cache.cache_map.copy().keys(), 
                                   "which_element": query.parameters.copy()})
        
    hits, misses = query.step()
    return hits - misses, cache_map_baseline

In [18]:
def compare_reward():
    env = setup()

    cache = env.cache.copy()
    query_baseline = Query(query_type="sequential", parameters=env.parameters.copy(), time=Time(0))
    
    cache.cache_map = {}
    query_baseline.set_query_cache(cache)
    
    cache_2 = env.cache.copy()
    query_baseline_random = Query(query_type="sequential", parameters=env.parameters.copy(), time=Time(0))
    
    cache_2.cache_map = {}
    query_baseline_random.set_query_cache(cache_2)
    
    reward_policy, cache_map_policy = evaluate_policy(env, q_table)
    reward_baseline, cache_map_baseline = get_baseline(query_baseline)
    
    
    reward_random, cache_map_random = get_baseline(query_baseline_random, "random")
    
    
    return {"policy": reward_policy, "reward_baseline": reward_baseline,
           "reward_random": reward_random, "parameters": env.parameters, "type": env.query_type}

In [19]:
results = []

for i in range(1000):
    res = compare_reward()
    results.append(res)

In [20]:
df_results = pd.DataFrame(results)

In [21]:
results[0]

{'policy': -9,
 'reward_baseline': -23,
 'reward_random': -11,
 'parameters': {'start': 9,
  'end': 14,
  'loop_size': 3,
  'current_position': 14,
  'current_counter': 3},
 'type': 'sequential'}

In [22]:
df_results[["reward_random", "reward_baseline", "policy"]]

,reward_random,reward_baseline,policy
0,-11,-23,-9
1,-5,-17,-9
2,-5,-11,-5
3,-3,-17,-9
4,-13,-23,-7
...,...,...,...
995,-5,-11,-1
996,-9,-17,-9
997,-5,-11,-9
998,-9,-23,-9


In [23]:
df = pd.DataFrame(cum_reward_plot)

In [24]:
df["r"] = df["reward"] / df["time"]

In [25]:
df["reward"] = (df["reward"]-df["reward"].mean())/df["reward"].std()

In [26]:
df[["reward", "time", "r"]]

,reward,time,r
0,-0.694470,15,-9.133333
1,-1.225154,17,-9.000000
2,-3.049382,22,-9.454545
3,-0.528631,15,-8.800000
4,0.565906,11,-9.000000
...,...,...,...
9994,0.400067,12,-8.666667
9995,-1.590000,18,-9.111111
9996,-0.329624,14,-9.000000
9997,-0.329624,14,-9.000000


In [27]:
(df["reward_baseline"] < df["policy"]).value_counts()

KeyError: 'reward_baseline'

In [ ]:
result = "Results/runs_{}_alpha_{}_gamma_{}_cache_{}.csv".format(number_of_runs, alpha, gamma, cache_size)

In [ ]:
df.to_csv(result)